# Model Training Pipeline

## Overview

This notebook trains multiclass classification models to generate i-MAP scores for distinguishing between multiple genetic/phenotypic groups using high-content imaging data. The pipeline implements in-silico multiplexing by integrating cellular profiles from multiple marker combinations imaged across separate plates.

**Workflow:**
1. **Configuration**: Set analysis parameters including screen name, marker combinations, and genetic classification scheme
2. **Data Loading**: Load single-cell imaging features from multiple marker sets using the `ImageScreenMultiAntibody` class
3. **Preprocessing**: Filter and normalize features (remove missing/constant features, filter outlier cells, standardize measurements)
4. **Model Training**: Train deep learning classifiers using the MAP (Molecular ALS Phenotype) analysis framework with cross-validation strategies (e.g., leave-one-out, sample splits)
5. **Model Saving**: Serialize trained models and analysis results for downstream evaluation and scoring

**Key Components:**
- **Model Architecture**: Multi-marker deep learning classifier with hierarchical encoders (marker-specific → cell-level → sample-level)
- **Training Strategy**: Two-phase training with per-marker early stopping followed by integrated model optimization
- **Sample Splits**: Configurable cross-validation (leave-one-out or fixed train/test splits) ensuring unbiased predictions on held-out samples
- **Output**: Trained models with predicted class probabilities and feature importances saved to disk

The trained models can be used to score new samples or evaluate classification performance on held-out data.

## 1. Configuration
Set analysis parameters including the screen name, analysis type (e.g., multiclass), marker subset filter, and specific marker combinations to analyze.

In [1]:
# ---- Parameters ----
SCREEN = "20250216_AWALS37_Full_screen_n96"
ANALYSIS = "multiclass_loocv"
ANTIBODY = ["HSP70/SOD1", "FUS/EEA1", "COX IV/Galectin3/atubulin"]
MARKER = "all"

In [2]:
# Parameters
SCREEN = "20250216_AWALS37_Full_screen_n96"
ANALYSIS = "multiclass_loocv"
ANTIBODY = ["HSP70/SOD1", "FUS/EEA1", "COX IV/Galectin3/atubulin"]
MARKER = "all"


Set random seeds across all libraries to ensure reproducible model training and evaluation.

In [3]:
import os
import random
import numpy as np
import torch

# ---- Set seeds for reproducibility ----
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Load analysis parameters from JSON configuration file and update with notebook-specific settings (screen name, marker combinations, feature filters).

In [4]:
from maps.screens import ImageScreenMultiAntibody
import json
from pathlib import Path


# --- Initialize parameters ---
pdir = Path("/home/kkumbier/maps/template_analyses/pipelines/params")
with open(pdir / f"{ANALYSIS}.json", "r") as f:
    params = json.load(f)

params["screen"] = SCREEN
params["antibodies"] = ANTIBODY

# Update marker if specified (e.g., to drop specific markers)
if MARKER != "all":
    fstr = params["preprocess"]["drop_feature_types"]["feature_str"]
    fstr += f"|^.*{MARKER}.*$"
    params["preprocess"]["drop_feature_types"]["feature_str"] = fstr

## 2. Data Loading & 3. Preprocessing
Load single-cell imaging features from multiple marker combinations and apply preprocessing steps: feature filtering, outlier removal, missing value imputation, and normalization.

In [5]:
# Load and process screens for train / test
screen = ImageScreenMultiAntibody(params)
screen.load(antibody=params["antibodies"])

print("Processing data...")
screen.preprocess()
assert screen.data is not None, "Loading failed"

for ab in params["antibodies"]:
    print(f"Marker set: {ab}")
    print(f"Data: {screen.data[ab].shape}")

Processing data...


Preprocessing complete
Marker set: HSP70/SOD1
Data: (86124, 311)
Marker set: FUS/EEA1
Data: (87657, 315)
Marker set: COX IV/Galectin3/atubulin
Data: (83045, 312)


## 4. Model Training
Train multiclass classification models using the MAP analysis framework. The `fit()` method executes the complete training pipeline including cross-validation, model fitting, and prediction generation on held-out samples.

In [6]:
from maps.analyses import MAP
map_analysis = MAP(screen)
map_analysis.fit()

---Training hold-out: ALS20---


Starting cell-level training...


Cell Epoch 1/100, Overall Loss: 1.1849, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1890, Acc: 0.3458 
  FUS/EEA1 - Loss: 1.1695, Acc: 0.3485 
  HSP70/SOD1 - Loss: 1.1962, Acc: 0.3485 


Cell Epoch 2/100, Overall Loss: 1.1284, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1285, Acc: 0.4104 
  FUS/EEA1 - Loss: 1.0913, Acc: 0.4150 
  HSP70/SOD1 - Loss: 1.1654, Acc: 0.3567 


Cell Epoch 3/100, Overall Loss: 1.1042, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1143, Acc: 0.4248 
  FUS/EEA1 - Loss: 1.0947, Acc: 0.4206 
  HSP70/SOD1 - Loss: 1.1038, Acc: 0.4154 


...

Line Epoch 17/100, Loss: 0.6237


Line Epoch 18/100, Loss: 0.6152


Line Epoch 19/100, Loss: 0.6721


Line Epoch 20/100, Loss: 0.6436


Line Epoch 21/100, Loss: 0.6834
Early stopping line training at epoch 21
Training completed!


## 5. Model Saving
Serialize the trained MAP analysis object (including fitted models, predictions, and feature importances) along with configuration parameters to disk for downstream evaluation.

In [7]:
import pickle

ab_string = "_".join(ANTIBODY).replace("/", "-")
output_dir = Path(params.get("result_dir")) / params.get("screen")
with open(output_dir / f"{ANALYSIS}-{ab_string}-{MARKER}.pkl", "wb") as f:
    pickle.dump({"analysis": map_analysis, "params": params}, f)